Lung nodule characterization <br/>
Copyright (C) 2017 Therapixel (Pierre Fillard).

Input data: LIDC (https://wiki.cancerimagingarchive.net/display/Public/LIDC-IDRI) with
nodule sizes (http://www.via.cornell.edu/lidc/). <br/>
Beforehant, all series shall be converted to a volumetric format (ITK MHD). Filenames
shall match the UID of each series. Binaries in the tools/ folder can be used for that 
(dataImporter, seriesExporter).

Annotation files are provided in csv format. Those depict positions (in real world
coordinates) from the series identified by the series UID (globally unique) where
nodules can be found, along with their characterization (inc. size).

This notebook will guide you through the process of training a deep net to classify
nodules vs non-nodules. The following steps are involved:
- data conversion: all annotations are turned into h5 arrays by extracting a patch
of size 64x64x64 around each position. Images are all resampled to have the same
voxel size of 0.625x0.625x0.625.
- model training: 4xGPUs were used to train this model using data-parallelism. 

To improve detection accuracy, 2 models need to be trained with a different seed each
time to split the dataset into training and validation. The final prediction probability
of wether a patch is a nodule or not is the average of each individual prediction of
each model.

In [ ]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
from datetime import datetime
import os.path
import glob
import time
from time import sleep
import sys
sys.path.append('../')
sys.path.append('../../')
import subprocess
import shutil

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected, convolution2d, flatten, batch_norm, max_pool2d, dropout, l2_regularizer
from tensorflow.python.ops.nn import relu, elu, relu6, relu1, sigmoid, tanh, softmax
from tensorflow.python.ops import variable_scope
import h5py as h5
import numpy as np
import lidc as lidc
import TherapixelDL.image as tpxdli
from six.moves import xrange
import scipy as sp
from scipy import ndimage
import csv
import SimpleITK as sitk

import matplotlib
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
import importlib
importlib.reload(lidc)
importlib.reload(tpxdli)

In [ ]:
def readCSV(filename):
    lines = []
    with open(filename, 'r') as f:
        csvreader = csv.reader(f)
        for line in csvreader:
            lines.append(line)
    return lines

In [ ]:
def nodule_csv_to_h5(nodule_file, data_directory, output_file):
    nodules = readCSV(nodule_file)
    nodules = nodules[1:] #skip header
    nodule_count = len(nodules)
    
    patch_size = 64
    patches = np.zeros(shape=(nodule_count,patch_size,patch_size,patch_size), dtype=np.float32)
    nodule_type = np.zeros(shape=(nodule_count), dtype=np.int32)
    num_reviews = np.zeros(shape=(nodule_count), dtype=np.int32)
    is_large = np.zeros(shape=(nodule_count), dtype=np.int32)
    subtlety = np.zeros(shape=(nodule_count), dtype=np.float32)
    internalStructure = np.zeros(shape=(nodule_count), dtype=np.int32)
    calcification = np.zeros(shape=(nodule_count), dtype=np.int32)
    sphericity = np.zeros(shape=(nodule_count), dtype=np.float32)
    margin = np.zeros(shape=(nodule_count), dtype=np.float32)
    lobulation = np.zeros(shape=(nodule_count), dtype=np.float32)
    spiculation = np.zeros(shape=(nodule_count), dtype=np.float32)
    texture = np.zeros(shape=(nodule_count), dtype=np.float32)
    malignancy = np.zeros(shape=(nodule_count), dtype=np.float32)
    diameter = np.zeros(shape=(nodule_count), dtype=np.float32)
    
    target_spacing = [0.625,0.625,0.625]

    pad_offset = patch_size//2

    current_seriesuid = ''

    for index in range(1,len(nodules)):
        nodule = nodules[index]
        seriesuid = nodule[0]

        # find series
        series_filename = data_directory + '/' + seriesuid + '.mhd'
        if (not os.path.isfile(series_filename)):        
            print('series not found:', seriesuid)
            continue

        if (seriesuid != current_seriesuid):
            print('reading / resampling series:', seriesuid)
            itk_image = sitk.ReadImage(series_filename)
            volume_orig, origin, spacing, orientation = lidc.parse_itk_image(itk_image)

            # resample using itk
            padding_value = volume_orig.min()
            img_z_orig, img_y_orig, img_x_orig = volume_orig.shape
            img_z_new = int(np.round(img_z_orig*spacing[2]/target_spacing[2]))
            img_y_new = int(np.round(img_y_orig*spacing[1]/target_spacing[1]))
            img_x_new = int(np.round(img_x_orig*spacing[0]/target_spacing[0]))

            itk_image_resampled = lidc.resample_itk_image(itk_image, [img_x_new,img_y_new,img_z_new], target_spacing, int(padding_value))
            volume, _, _, _ = lidc.parse_itk_image(itk_image_resampled)
            volume = volume.astype(np.float32)
            volume = lidc.normalizePlanes(volume)
            volume = np.pad(volume, ((pad_offset,pad_offset), (pad_offset,pad_offset), (pad_offset,pad_offset)), 
                            'constant', constant_values=((0, 0),(0, 0),(0, 0)))    
            current_seriesuid = seriesuid
            print('done')

        x = float(nodule[1])
        y = float(nodule[2])
        z = float(nodule[3])

        k,j,i = lidc.worldToVoxelCoord([x,y,z], origin, target_spacing, orientation)
        k = int(round(k))
        j = int(round(j))
        i = int(round(i))
        k = min(max(0, k), volume.shape[0]-patch_size)
        j = min(max(0, j), volume.shape[1]-patch_size)
        i = min(max(0, i), volume.shape[2]-patch_size)

        patches[index-1] = volume[k:k+patch_size,j:j+patch_size,i:i+patch_size]
        nodule_type[index-1] = int(nodule[4])
        num_reviews[index-1] = int(nodule[5])
        is_large[index-1] = int(nodule[6])    
        subtlety[index-1] = float(nodule[7])
        internalStructure[index-1] = int(nodule[8])
        calcification[index-1] = int(nodule[9])
        sphericity[index-1] = float(nodule[10])
        margin[index-1] = float(nodule[11])
        lobulation[index-1] = float(nodule[12])
        spiculation[index-1] = float(nodule[13])
        texture[index-1] = float(nodule[14])
        malignancy[index-1] = float(nodule[15])
        diameter[index-1] = float(nodule[16])

    # create h5 dataset
    h5_file = h5.File(output_file, 'w')
    h5_file.create_dataset('PATCHES', data = patches, dtype=np.float32)
    h5_file.create_dataset('LABELS', data = nodule_type, dtype=np.int32)
    h5_file.create_dataset('REVIEWS', data = num_reviews, dtype=np.int32)
    h5_file.create_dataset('IS_LARGE', data = is_large, dtype=np.int32)
    h5_file.create_dataset('SUBTLETY', data = subtlety, dtype=np.float32)
    h5_file.create_dataset('INTERNALSTRUCTURE', data = internalStructure, dtype=np.int32)
    h5_file.create_dataset('CALCIFICATION', data = calcification, dtype=np.int32)
    h5_file.create_dataset('SPHERICITY', data = sphericity, dtype=np.float32)
    h5_file.create_dataset('MARGIN', data = margin, dtype=np.float32)
    h5_file.create_dataset('LOBULATION', data = lobulation, dtype=np.float32)
    h5_file.create_dataset('SPICULATION', data = spiculation, dtype=np.float32)
    h5_file.create_dataset('TEXTURE', data = texture, dtype=np.float32)
    h5_file.create_dataset('MALIGNANCY', data = malignancy, dtype=np.float32)
    h5_file.create_dataset('DIAMETER', data = diameter, dtype=np.float32)
    h5_file.close()

In [ ]:
# data conversion - once for all

nodule_file = 'lidc_nodules_with_size.csv'
data_directory = '/media/data/LIDC/LIDC-MHD/'
output_file = 'lidc_nodules_with_size.h5'

nodule_csv_to_h5(nodule_file, data_directory, output_file)

In [ ]:
lidc_filepath = 'lidc_nodules_with_size.h5'

train_data_list = []
train_target_list = []
validation_data_list = []
validation_target_list = []

val_ratio = 0.1

REVIEWS_MIN=1.
REVIEWS_MAX = 4.
SUBTLETY_MIN = 1.
SUBTLETY_MAX = 5.
SPHERICITY_MIN = 1.
SPHERICITY_MAX = 5.
MARGIN_MIN = 1.
MARGIN_MAX = 5.
LOBULATION_MIN = 1.
LOBULATION_MAX = 5.
SPICULATION_MIN = 1.
SPICULATION_MAX = 5.
TEXTURE_MIN = 1.
TEXTURE_MAX = 5.
MALIGNANCY_MIN = 1.
MALIGNANCY_MAX = 5.
DIAMETER_MIN = 0.
DIAMETER_MAX = 20.

# read nodules
h5_file = h5.File(lidc_filepath, 'r')
patches = h5_file['PATCHES'][...]
labels = h5_file['LABELS'][...]
is_large = h5_file['IS_LARGE'][...]
malignancy = h5_file['MALIGNANCY'][...]
diameter = h5_file['DIAMETER'][...]
h5_file.close()

malignancy = (malignancy.astype(np.float32)-MALIGNANCY_MIN)/(MALIGNANCY_MAX-MALIGNANCY_MIN)
diameter = (diameter.astype(np.float32)-DIAMETER_MIN)/(DIAMETER_MAX-DIAMETER_MIN)

indices = np.arange(patches.shape[0])
indices = indices[(labels==1) & (is_large==1) & (diameter>0.)] # only large nodules with size

# concat
malignancy = malignancy.reshape(tuple(malignancy.shape)+tuple([1]))
diameter = diameter.reshape(tuple(diameter.shape)+tuple([1]))

targets = np.concatenate((malignancy,diameter), axis=1)

np.random.shuffle(indices)
count = len(indices)
val_count = int(count * val_ratio)
train_count = count - val_count
train_patches = patches[indices[val_count:]]
train_targets = targets[indices[val_count:]]
val_patches = patches[indices[:val_count]]
val_targets = targets[indices[:val_count]]
    
train_data_list.append(train_patches)
train_target_list.append(train_targets)
validation_data_list.append(val_patches)
validation_target_list.append(val_targets)

In [ ]:
print('is_large', is_large[indices].min(), is_large[indices].max())
print('malignancy', malignancy[indices].min(), malignancy[indices].max())
print('diameter', diameter[indices].min(), diameter[indices].max())

In [ ]:
# hyperameters of the model
channels = 1
scalings=None #np.array([1.0, 1.5])
offsets=None #np.array([0.0, -0.05])
depth = 64
height = 64
width = 64
num_gpus = 4
batch_size = 32
patch_size = 64
gpu_mem_ratio = 1.0
num_outputs = 2

In [ ]:
shift_range = 0.0
image_gen_3d = tpxdli.ImageDataGenerator3D(rotation_range=180.0, width_shift_range=shift_range, height_shift_range=shift_range, depth_shift_range=shift_range,
                                           shear_range=0.1, zoom_range=np.array([0.95,1.05], dtype=np.float32),
                                           horizontal_flip=True, vertical_flip=True, depth_flip=True,
                                           windowing_scale_range=0.0, windowing_intercept_range=0.0,
                                           dim_ordering = 'tf')
# do not augment validation batch to simulate real-life data
image_gen_3d_val = tpxdli.ImageDataGenerator3D(rotation_range=0.0, width_shift_range=0.0, height_shift_range=0.0, depth_shift_range=0.0,
                                           shear_range=0.0, zoom_range=np.array([1.0,1.0], dtype=np.float32), horizontal_flip=False, vertical_flip=False, depth_flip=False,
                                           dim_ordering = 'tf')

In [ ]:
def train(train_data, train_targets, validation_data, validation_targets, lr_scheme, num_gpus=1, num_epochs=100,
          output_dir='', prev_model=''):
        
    # reset graph first
    tf.reset_default_graph()
    
    with tf.Graph().as_default(), tf.device('/cpu:0'):
        global_step = tf.contrib.framework.get_or_create_global_step()
        
        is_training = tf.placeholder(tf.bool, shape=[], name='is_training')
    
        # Setting up placeholder, this is where your data enters the graph!
        x_pl = tf.placeholder(tf.float32, shape=(None, height, width, depth, channels), name='data_x')
        y_pl = tf.placeholder(tf.float32, shape=(None, num_outputs), name='data_y')
    
        # defining our optimizer
        learning_rate = tf.placeholder(tf.float32, shape=[])
        
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

        # Calculate the gradients for each model tower.
        tower_grads = []    
        losses = []
        y = []
    
        x_splits = tf.split(x_pl, num_or_size_splits=num_gpus)
        y_splits = tf.split(y_pl, num_or_size_splits=num_gpus)
    
        with tf.variable_scope(tf.get_variable_scope()) as scope:
            for i in range(num_gpus):
                with tf.device('/gpu:%d' % i):
                    with tf.name_scope('tower_%d' % (i)) as scope:

                        preds, _ = lidc.inference_all_scores(x_splits[i], 
                                                          num_outputs=num_outputs,
                                                          is_training=is_training)
                        preds = tf.squeeze(preds)
                        l = lidc.l2_loss(preds=preds, values=y_splits[i], with_regularization=False)

                        # Reuse variables for the next tower.
                        tf.get_variable_scope().reuse_variables()

                        # Calculate the gradients for the batch of data
                        grads = optimizer.compute_gradients(l)

                        # Keep track of the gradients across all towers.
                        tower_grads.append(grads)
                        losses.append(l)
                        y.append(preds)

        # We must calculate the mean of each gradient. Note that this is the
        # synchronization point across all towers.
        if (num_gpus>1):
            grads = lidc.average_gradients(tower_grads)    
        else:
            grads = tower_grads[0]
    
        # Apply the gradients to adjust the shared variables.
        apply_gradient_op = optimizer.apply_gradients(grads)
    
        # Track the moving averages of all trainable variables.
        variable_averages = tf.train.ExponentialMovingAverage(lidc.MOVING_AVERAGE_DECAY, global_step)
        variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
        with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
            train_op = tf.no_op(name='train')
            
        # Restore the moving average version of the learned variables for eval.
        variable_averages = tf.train.ExponentialMovingAverage(lidc.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore, max_to_keep=None)
        
        # restricting memory usage, TensorFlow is greedy and will use all memory otherwise
        gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_mem_ratio)
        # initialize the Session
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts, 
                                                allow_soft_placement=True, 
                                                log_device_placement=True)) # allow_soft_placement=True needed to make batch_normalization work accross GPU
            
        sess.run(tf.global_variables_initializer())
        if (prev_model):
            print('restoring model', prev_model)
            saver.restore(sess, prev_model)            
    
    n_train_samples = train_count
    train_capacity = batch_size * num_gpus
    num_batches_train = n_train_samples // train_capacity
    
    train_iterator_3d = image_gen_3d.flowList(X=train_data, 
                                              Y=train_targets, 
                                              batch_size=train_capacity,
                                              balance=False,
                                              shuffle=True, 
                                              output_depth=patch_size, 
                                              output_rows=patch_size, 
                                              output_cols=patch_size,
                                              num_output_channels=channels,
                                              scalings=scalings,
                                              offsets=offsets)
    
    val_capacity = batch_size * num_gpus
    val_iterator_3d = image_gen_3d_val.flowList(X=validation_data, 
                                                Y=validation_targets,
                                                batch_size=val_capacity,
                                                balance=False,
                                                shuffle=False,
                                                output_depth=patch_size, 
                                                output_rows=patch_size, 
                                                output_cols=patch_size,
                                                num_output_channels=channels,
                                                scalings=scalings,
                                                offsets=offsets)
    
    n_val_samples = val_count
    num_batches_valid = n_val_samples // val_capacity    

    print('training with parameters:\n\t- train capacity: %d\n\t- val capacity: %d\n\t- batch size: %d\n\t- patch size: %d\n\t'\
          '- num gpu: %d\n\t- num epochs: %d\n\t- previous model: %s' % (n_train_samples, n_val_samples, batch_size, patch_size,
                                                                         num_gpus, num_epochs, prev_model))           
    
    print('number of training batches per epoch', num_batches_train)
    print('number of validation batches per epoch', num_batches_valid)
    
    lr = -1
    best_val_loss = -1.
    train_loss = 0.
    valid_loss = 0.
    
    train_queue = tpxdli.QueuedIterator(train_iterator_3d, num_batches_train)
    val_queue = tpxdli.QueuedIterator(val_iterator_3d, num_batches_valid)
    
    try:
        # init best_val_loss before training
        val_queue.produce()
        for i in range(num_batches_valid):
            (batch_val_x, batch_val_y) = val_queue.get_queue().get()            
            feed_dict_eval = {
                x_pl: batch_val_x,
                y_pl: batch_val_y,
                is_training: False
            }
            fetches_eval = [losses]
            # running the validation
            res = sess.run(fetches=fetches_eval, feed_dict=feed_dict_eval)
            # collecting and storing predictions
            cur_loss = np.sum(res[0])
            if i==0:
                valid_loss = cur_loss/(num_gpus)
            else:
                valid_loss = valid_loss*i/(i+1) + cur_loss/(num_gpus*(i+1))
            val_queue.get_queue().task_done()
            sys.stdout.write('\rValidation. batch: %d/%d. loss: %f'%(i+1,num_batches_valid,valid_loss))
            sys.stdout.flush()
            sleep(1)
            
        best_val_loss = valid_loss
        print('\nInitial validation loss is: %f'%(best_val_loss))                
    
        for epoch in range(num_epochs):
            
            if (lr != lr_scheme[epoch]):
                lr = lr_scheme[epoch]
                print('using lr', lr)
        
            t0 = time.time()                
                        
            train_queue.produce()
            for i in range(num_batches_train):                
                (batch_train_x, batch_train_y) = train_queue.get_queue().get()                
                feed_dict_train = {
                    x_pl: batch_train_x,
                    y_pl: batch_train_y,
                    is_training: True, 
                    learning_rate: lr
                }
                fetches_train = [train_op, losses]
                res = sess.run(fetches=fetches_train, feed_dict=feed_dict_train)
                cur_loss = np.sum(res[1])
                if i==0:
                    train_loss = cur_loss/(num_gpus)
                else:
                    train_loss = train_loss*i/(i+1) + cur_loss/(num_gpus*(i+1))                                
                train_queue.get_queue().task_done()
                sys.stdout.write('\rTraining. batch: %d/%d, loss: %f'%(i+1,num_batches_train,train_loss))
                sys.stdout.flush()
                sleep(1)
                    
            t1 = time.time()
            epoch_time = t1 - t0
        
            sys.stdout.write("\n")
        
            val_queue.produce()
            #p_preds = []
            #p_true = []
            for i in range(num_batches_valid):
                (batch_val_x, batch_val_y) = val_queue.get_queue().get()                
                feed_dict_eval = {
                    x_pl: batch_val_x,
                    y_pl: batch_val_y,
                    is_training: False
                }
                fetches_eval = [losses, preds]
                # running the validation
                res = sess.run(fetches=fetches_eval, feed_dict=feed_dict_eval)
                # collecting and storing predictions
                cur_loss = np.sum(res[0])
                if i==0:
                    valid_loss = cur_loss/num_gpus
                else:
                    valid_loss = valid_loss*i/(i+1) + cur_loss/(num_gpus*(i+1))
                #p_preds.append(res[1])
                #p_true.append(batch_val_y)
                val_queue.get_queue().task_done()                
                sys.stdout.write('\rValidation. batch: %d/%d, loss: %f'%(i+1,num_batches_valid,valid_loss))
                sys.stdout.flush()
                sleep(1)
                            
            sys.stdout.write("\n")
            
            #print('preds', np.concatenate(p_preds))
            #print('truth', np.concatenate(p_true))

            print ("Epoch %i: train loss %e, valid loss %f, epoch time %.2f s " \
            % (epoch+1, train_loss, valid_loss, epoch_time))
        
            if (best_val_loss<0):
                best_val_loss = valid_loss
            
            if ((best_val_loss>=0) and (valid_loss<best_val_loss)):
                print('val loss improved from %f to %f, saving model' % (best_val_loss, valid_loss))
                best_val_loss = valid_loss
                if (output_dir):
                    filename = output_dir + 'best_model_loss'
                    print('saving model to file:',filename)
                    saver.save(sess, filename)
                    
            if ((epoch%10)==0):
                saver.save(sess, output_dir+'checkpoint-epoch-%.3d-loss-%.4f'%(epoch,valid_loss))
                
            epoch += 1
            
            train_loss = 0.
            valid_loss = 0.
            
    except KeyboardInterrupt:        
        pass
    
    train_queue.get_queue().join()
    val_queue.get_queue().join()
    
    sess.close()

In [ ]:
num_epochs = 300
prev_model = ''

# to run:
# - once 300 epochs with AdamOptimizer and lr=1e-3 then 1e-4 after 100 epochs
# - once 300 epochs with SGB and lr=1e-5
# Learn 2 models as above and average predictions

output_directory = 'model_1/'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    
lr_scheme = np.zeros(shape=(num_epochs), dtype=np.float32)
lr = 1e-3 #1e-4
lr_decay = 10.
lr_scheme[0:100] = lr
lr /= lr_decay
lr_scheme[100:] = lr
    
# train model    
train(train_data=train_data_list, 
      train_targets=train_target_list,
      validation_data=validation_data_list, 
      validation_targets=validation_target_list,
      lr_scheme=lr_scheme,
      num_gpus=num_gpus,
      num_epochs=num_epochs,      
      output_dir=output_directory,
      prev_model=prev_model)